In [ ]:
#로컬에 설치한 AgensGraph를 python으로 접근하여 테이블 데이터 생성하기기
import psycopg2
from faker import Faker
import random
from datetime import timedelta

# PostgreSQL 연결 정보
conn = psycopg2.connect(
    dbname="agens",
    user="postgres",
    password="agens",
    host="localhost",
    port="5455"
)

cur = conn.cursor()
faker = Faker()

# 가상 데이터 생성
transaction_types = ['deposit', 'withdrawal', 'transfer']

for _ in range(1000):
    account_id = random.randint(1, 100)  # 예시: 계좌 ID가 1~100 사이 있다고 가정
    transaction_type = random.choice(transaction_types)
    amount = round(random.uniform(10.00, 10000.00), 2)
    transaction_date = faker.date_time_between(start_date='-1y', end_date='now')
    description = faker.sentence(nb_words=6)

    cur.execute("""
        INSERT INTO transactions (account_id, transaction_type, amount, transaction_date, description)
        VALUES (%s, %s, %s, %s, %s)
    """, (account_id, transaction_type, amount, transaction_date, description))

# 커밋 및 종료
conn.commit()
cur.close()
conn.close()

print("1000개의 거래 데이터가 삽입되었습니다.")

In [1]:
# 랭채인에 있는 오픈AI 라이브러리리
from langchain_openai import ChatOpenAI

In [3]:
# 랭체인의 기본 질의 응답답
llm = ChatOpenAI(model="gpt-4o-mini")
answer = llm.invoke('OECD국가는 몇개이며 한국은 경제규 몇위인지?')
answer

AIMessage(content='OECD(경제협력개발기구)는 38개의 회원국으로 구성되어 있습니다. 한국은 OECD 회원국 중 하나이며, 경제규모 면에서는 대체로 10위권 내에 위치해 있습니다. 구체적인 순위는 매년 경제 성장률 및 통계에 따라 변동이 있을 수 있으므로, 최신 자료를 통해 확인하는 것이 좋습니다. 예를 들어, 2023년 기준으로 한국의 경제규모는 대략 10위로 평가되고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 22, 'total_tokens': 130, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'id': 'chatcmpl-Bbdv87hdXLI2lXhjOTxZW3AvE3H0U', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--1e0f4dab-ae5b-4010-9168-ae9c1e452688-0', usage_metadata={'input_tokens': 22, 'output_tokens': 108, 'total_tokens': 130, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning

In [11]:
# 랭체인 프롬프트 관린 라이브러리리
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("너는 천문학 분야 전문가로써 질문에 답변을 해줘 <Question>:{input}")

chain = prompt | llm
chain.invoke({"input" : "지구의 자전 주기는?"})

AIMessage(content="지구의 자전 주기는 약 24시간입니다. 정확히 말하면, 지구는 한 번 자전하는 데 약 23시간 56분 4초가 걸립니다. 이를 '항성일'이라고 하며, 일반적으로 우리가 사용하는 24시간은 태양이 하늘에서 이동하는 것을 기준으로 하는 '태양일'을 나타냅니다. 태양일은 약간 길어지는 이유는 지구가 태양 주위를 공전하면서 회전하기 때문입니다.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 34, 'total_tokens': 142, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'id': 'chatcmpl-BZseqb7LffJ2JT2lJigKXwVEZ9sMZ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d3eb2ba6-aa59-49c5-886e-fb8d0f95c676-0', usage_metadata={'input_tokens': 34, 'output_tokens': 108, 'total_tokens': 142, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser
chain.invoke({"input" : "지구의 자전 주기는?"})

'지구의 자전 주기는 약 24시간입니다. 좀 더 정확히 말하면, 평균적으로 23시간 56분 4초 정도입니다. 이 시간을 “자전일”이라고 하며, 이는 지구가 한 번 자전하는 데 걸리는 시간입니다. 그러나 우리가 일반적으로 사용하는 24시간은 평균적인 태양일을 기준으로 하며, 태양이 다시 같은 위치에 나타나는 시간을 기준으로 합니다.'

In [13]:
prompt1 = ChatPromptTemplate.from_template("translates {Korean_word} to English and one word answer")
prompt2 = ChatPromptTemplate.from_template("explain {English_word} using oxford dictionary to me in korean ")

llm = ChatOpenAI(model = 'gpt-4o-mini')
chain1 = prompt1 | llm | StrOutputParser()
chain1.invoke({"Korean_word" : "미래"})

'Future'

In [14]:
chain2 = (
    {"English_word" : chain1} | prompt2 | llm | StrOutputParser()
          )

chain2.invoke({"Korean_word" : "미래"})

'"Future"라는 단어는 옥스포드 사전에서 다음과 같이 정의됩니다:\n\n1. **미래**: 시간이 아직 오지 않은 시점, 즉 현재와 과거 사이에 위치한 시간.\n2. **미래의 일들**: 앞으로 발생할 가능성이 있는 사건이나 상황들.\n\n한국어로 설명하자면, \'미래\'는 현재와 과거 사이에 있으며, 앞으로 일어날 가능성이 있는 상황이나 사건들을 의미합니다. 예를 들어, 우리의 계획이나 꿈, 예상되는 변화 등이 모두 미래와 관련이 있습니다.'

In [13]:
# 라이브러리 임포트
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import init_chat_model
from langchain.schema.output_parser import StrOutputParser

In [ ]:
# langchain의 기존 사용 구조

# 1. 컴포넌트 정의
prompt = ChatPromptTemplate.from_template("전문가의 관점으로 {input}을 쉽고 간단히 설명해줘")
model = init_chat_model("gpt-4o-mini", model_provider="openai")
output_parser = StrOutputParser()

# 2. 체인생성
chain = prompt | model | output_parser

# 3. 메소드 사용
result = chain.invoke({"input":"langchain공부법"})
print(result)

LangChain은 인공지능과 자연어 처리(NLP)를 활용하여 언어 모델을 개선하고 다양하게 활용할 수 있도록 돕는 프레임워크입니다. LangChain을 공부하는 방법은 다음과 같이 정리할 수 있습니다.

### 1. 기본 개념 이해하기
- **인공지능과 자연어 처리(NLP)**: 기초적인 AI와 NLP의 원리를 이해합니다. 특히, 텍스트 데이터의 처리 방법, 언어 모델의 작동 방식 등을 학습합니다.

### 2. LangChain의 아키텍처 이해하기
- **구성 요소**: LangChain의 핵심 구성 요소(예: 프롬프트 관리, 체인, 에이전트 등)를 이해합니다. 각 요소의 역할과 실제 사용 사례를 조사합니다.

### 3. 공식 문서 및 튜토리얼 활용하기
- **공식 문서**: LangChain의 [공식 문서](https://langchain.readthedocs.io/)를 정독하며 구체적인 함수와 사용법을 익힙니다.
- **튜토리얼**: 초기부터 중급 수준까지의 튜토리얼을 따라 해보며, 샘플 코드와 실습을 통해 경험을 쌓습니다.

### 4. 실습 프로젝트 진행하기
- **작은 프로젝트**: 간단한 프로젝트를 설정하여 LangChain을 활용해 보세요. 예를 들어, 챗봇, 텍스트 요약기 등을 만들어 볼 수 있습니다.
- **커뮤니티 참여**: GitHub, 포럼, Discord 등의 커뮤니티에 참여하여 질문하고, 다른 사람들의 프로젝트를 통해 영감을 얻습니다.

### 5. 고급 기능 탐구하기
- **커스터마이징**: 기본적인 사용법을 익힌 후, LangChain의 API를 활용하여 자신만의 커스터마이징된 애플리케이션을 개발해봅니다.
- **성능 개선 및 최적화**: 모델의 성능을 개선하기 위한 다양한 기법들을 비교하고 적용해보는 과정도 중요합니다.

### 6. 지속적인 학습
- **최신 트렌드**: AI와 NLP 분야는 빠르게 변화합니다. 최신 연구 동향, 사용자 사례 등을 지속적으로 학습하여 지식을 업데이트합니다.

이렇게 단계별로 접근하면 Lang

In [20]:
# 4-1. 메서드사용
inputs = ["LangGraph","LangSmith","LangChain"]
results = chain.batch([{"input":i} for i in inputs])
results

['LangGraph은 언어와 관련된 다양한 데이터를 시각적으로 나타내고 분석할 수 있는 도구입니다. 더 쉽게 설명하자면, 텍스트와 단어의 관계를 그래프로 표현하여 언어의 구조와 의미를 이해하는 데 도움을 주는 플랫폼입니다.\n\n1. **언어 데이터 시각화**: LangGraph은 단어, 문장, 또는 문서 간의 관계를 그래프 형태로 나타냅니다. 이를 통해 사용자는 특정 단어가 다른 단어와 어떻게 연결되어 있는지 쉽게 파악할 수 있습니다.\n\n2. **관계 분석**: 예를 들어, 단어의 동의어, 대칭어, 그리고 문맥에 따라 어떻게 쓰이는지를 분석하여 언어의 뉘앙스를 이해하게 합니다.\n\n3. **사용자 친화적인 인터페이스**: 복잡한 NLP(자연어 처리) 기술 없이도 그래프를 통해 데이터를 탐색하고 언어의 패턴을 발견할 수 있는 직관적인 인터페이스를 제공합니다.\n\n4. **교육 및 연구 도구**: 언어학, 문학 연구, 교육 등의 분야에서 유용하게 사용할 수 있습니다. 학습자가 언어 구조를 이해하는 데 큰 도움이 됩니다.\n\n이렇게 LangGraph은 언어의 패턴을 시각적으로 쉽게 이해하고, 활용할 수 있도록 돕는 도구입니다.',
 'LangSmith는 주로 자연어 처리(NLP)와 관련된 도구 또는 플랫폼으로, 사용자들이 다양한 언어 모델을 쉽게 구축하고 관리할 수 있도록 돕는 솔루션입니다. 이 플랫폼은 개발자나 데이터 과학자가 복잡한 알고리즘이나 수학적 모델링 없이도 언어 모델을 실험하고 적용할 수 있는 환경을 제공합니다.\n\nLangSmith의 주요 기능은 다음과 같습니다:\n\n1. **모델 훈련**: 사용자들이 데이터를 입력하여 자신만의 언어 모델을 훈련시킬 수 있도록 지원합니다.\n\n2. **프로토타이핑**: 빠르게 프로토타입을 만들고 테스트해 볼 수 있는 기능을 제공하여, 아이디어를 신속하게 검증할 수 있습니다.\n\n3. **커스터마이징**: 사용자 요구에 맞춰 모델을 커스터마이즈할 수 있는 옵션이 있어, 특정 업무나 도메인에 적합한 솔

In [23]:
print(results[0])
print(results[1])
print(results[2])

LangGraph은 언어와 관련된 다양한 데이터를 시각적으로 나타내고 분석할 수 있는 도구입니다. 더 쉽게 설명하자면, 텍스트와 단어의 관계를 그래프로 표현하여 언어의 구조와 의미를 이해하는 데 도움을 주는 플랫폼입니다.

1. **언어 데이터 시각화**: LangGraph은 단어, 문장, 또는 문서 간의 관계를 그래프 형태로 나타냅니다. 이를 통해 사용자는 특정 단어가 다른 단어와 어떻게 연결되어 있는지 쉽게 파악할 수 있습니다.

2. **관계 분석**: 예를 들어, 단어의 동의어, 대칭어, 그리고 문맥에 따라 어떻게 쓰이는지를 분석하여 언어의 뉘앙스를 이해하게 합니다.

3. **사용자 친화적인 인터페이스**: 복잡한 NLP(자연어 처리) 기술 없이도 그래프를 통해 데이터를 탐색하고 언어의 패턴을 발견할 수 있는 직관적인 인터페이스를 제공합니다.

4. **교육 및 연구 도구**: 언어학, 문학 연구, 교육 등의 분야에서 유용하게 사용할 수 있습니다. 학습자가 언어 구조를 이해하는 데 큰 도움이 됩니다.

이렇게 LangGraph은 언어의 패턴을 시각적으로 쉽게 이해하고, 활용할 수 있도록 돕는 도구입니다.
LangSmith는 주로 자연어 처리(NLP)와 관련된 도구 또는 플랫폼으로, 사용자들이 다양한 언어 모델을 쉽게 구축하고 관리할 수 있도록 돕는 솔루션입니다. 이 플랫폼은 개발자나 데이터 과학자가 복잡한 알고리즘이나 수학적 모델링 없이도 언어 모델을 실험하고 적용할 수 있는 환경을 제공합니다.

LangSmith의 주요 기능은 다음과 같습니다:

1. **모델 훈련**: 사용자들이 데이터를 입력하여 자신만의 언어 모델을 훈련시킬 수 있도록 지원합니다.

2. **프로토타이핑**: 빠르게 프로토타입을 만들고 테스트해 볼 수 있는 기능을 제공하여, 아이디어를 신속하게 검증할 수 있습니다.

3. **커스터마이징**: 사용자 요구에 맞춰 모델을 커스터마이즈할 수 있는 옵션이 있어, 특정 업무나 도메인에 적합한 솔루션을 구현할 수 있습니다.

4. **사용

In [27]:
# 4-2. 매서드사용
stream = chain.stream({"input":"LangSmith"})
print("stream 결과:")
for chunk in stream:
    print(chunk, end="", flush=True)
print()

stream 결과:
LangSmith는 AI 언어 모델을 개발하고 관리하는 데 도움을 주는 도구입니다. 이 플랫폼은 주로 코딩 및 언어 처리 작업을 보다 효율적으로 수행할 수 있도록 설계되었습니다. 

주요 기능으로는 다음과 같은 것들이 있습니다:

1. **모델 훈련**: 사용자는 쉽게 AI 모델을 훈련시키고, 자신의 데이터를 기반으로 맞춤형 모델을 만들 수 있습니다.
2. **테스트 및 평가**: AI 모델의 성능을 테스트하고 평가할 수 있는 도구를 제공합니다.
3. **협업 기능**: 팀원들과의 협업을 통해 여러 사람과 함께 모델을 개발하고 개선할 수 있습니다.

따라서, LangSmith은 개발자나 데이터 과학자들이 보다 간편하게 AI 모델을 다루고, 이를 통해 다양한 언어 처리 작업을 수행할 수 있도록 돕는 플랫폼입니다.


## 프롬프트 작성 요령
- 질문은 명확하고 구체적
- 모델이 문맥을 이해할 수 있도록 필요한 배경 정보를 제공
- 핵심 정보에 초점을 맞추고, 불필요한 정보는 배제, 프롬프트가 길어지면 모델이 덜 중요한 부분에 집중하거나 상당한 영향을 받는 문제가 발생 가능
- 열린 질문을 통해 모델이 자세하고 풍부한 답변을 제공하도록 유도
- 얻고자 하는 정보나 결과의 유형을 정확하게 정의
- 대화의 맥락에 적합한 언어와 문체를 선택

In [4]:
template_text = "안녕하세요 제 이름은 {name}이고 , 나이는 {age}살입니다"
prompt_template = PromptTemplate.from_template(template_text)
filled_prompt = prompt_template.format(name = '김보생', age = '39')
filled_prompt

'안녕하세요 제 이름은 김보생이고 , 나이는 39살입니다'

In [8]:
combined_prompt = (prompt_template + prompt_template.from_template("\n\n아버지를 아버지라 부를 수 없습니다.") + "\n\n{language}로 번역해 주세요")
combined_prompt

PromptTemplate(input_variables=['age', 'language', 'name'], input_types={}, partial_variables={}, template='안녕하세요 제 이름은 {name}이고 , 나이는 {age}살입니다\n\n아버지를 아버지라 부를 수 없습니다.\n\n{language}로 번역해 주세요')

In [9]:
combined_prompt.format(name = "홍길동", age = '30', language = '영어')

'안녕하세요 제 이름은 홍길동이고 , 나이는 30살입니다\n\n아버지를 아버지라 부를 수 없습니다.\n\n영어로 번역해 주세요'

In [14]:
model = init_chat_model("gpt-4o-mini", model_provider="openai")
output_parser = StrOutputParser()

chain = combined_prompt | model | output_parser
chain.invoke({"name" : "홍길동", "age" : 30, "language":"english"})

'Hello, my name is Hong Gildong, and I am 30 years old. I cannot call my father "father."'